In [ ]:
# extract sanger male normal lines from tracker

from depmapomics.tracker import getTracker
tracker = getTracker()
sanger = tracker[(tracker['source'] == 'SANGER') & (tracker['primary_disease'].str.lower()=='normal')]
assert (sanger['datatype'] == 'wes').all()

sanger['sex'].value_counts()

In [ ]:
sanger = sanger[sanger.sex == 'Male']
sanger

In [ ]:
sanger = sanger[['internal_bam_filepath', 'internal_bai_filepath', 'participant_id']]
sanger = sanger.rename(columns={'internal_bam_filepath': 'hg38_analysis_ready_bam', 'internal_bai_filepath': 'hg38_analysis_ready_bam_index'})
sanger

In [ ]:
# import the above lines to terra workspace broad-firecloud-ccle/DepMap_WES_PONS_hg38
from depmapomics import loading
import dalmatian as dm

refworkspace = 'broad-firecloud-ccle/DepMap_WES_PONS_hg38-copy'
samplesetname = 'SANGER_XY'
samples = sanger

samples.index.name = 'sample_id'
refwm = dm.WorkspaceManager(refworkspace).disable_hound()
refwm.upload_samples(samples)

#creating a sample set
refwm.update_sample_set(sample_set_id=samplesetname,
                      sample_ids=samples.index)


In [ ]:
from genepy import terra

submission_id = refwm.create_submission("CNV_Somatic_Panel_Workflow_Agilent_XY", samplesetname)
await terra.waitForSubmission(refworkspace, submission_id)